# Large Language Model for Hydrogen Storage [Text Summarization]



---



In [ ]:
# Installing/updating necessary libraries using pip
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn
!pip install -U sentencepiece
!pip install -U urllib3
!pip install py7zr
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none

In [ ]:
import os

In [ ]:
# Importing necessary libraries
from datasets import DatasetDict, Dataset
import pandas as pd

# Reading data from CSV file into a pandas DataFrame
df = pd.read_csv("final_dataset.csv")

In [ ]:
df.shape

(515, 4)



---



# Preprocessing The Data

In [ ]:
# Importing the 're' module for regular expressions
import re


def preprocess_data(data):
    processed_data = data.copy()
    processed_data['Abstract'] = processed_data['Abstract'].apply(clean_text)
    return processed_data

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Preprocessing the data
df = preprocess_data(df)

In [ ]:
# Define features for each dataset split
features = ['Abstract','Summary',"Id"]

num_rows_train = 315
num_rows_validation = 100
num_rows_test = 100

# Define dataset splits
train_dataset = Dataset.from_pandas(df.head(num_rows_train))
validation_dataset = Dataset.from_pandas(df.sample(num_rows_validation))
test_dataset = Dataset.from_pandas(df.tail(num_rows_test + num_rows_validation).head(num_rows_test))

# Create a DatasetDict object
hydro = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

In [ ]:
hydro['train']

Dataset({
    features: ['Id', 'Abstract', 'Summary', 'Unnamed: 3'],
    num_rows: 315
})

In [ ]:
# Importing the pipeline function from the transformers library for easy access to pre-trained models
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Evaluation Of The Model Before Finetuning

In [ ]:
# Importing the random module for generating random numbers
import random
start, end = 0, 100

random_numbers = [random.randint(start, end) for _ in range(5)]
random_numbers

[45, 71, 10, 68, 12]

In [ ]:
# Extracting reference summaries from the validation dataset based on randomly generated indices
reference_summaries = [0]*5
j = 0

for i in random_numbers:
  reference_summaries[j] = hydro["validation"][i]["Summary"]
  j += 1

In [ ]:
# Generating summaries using the pipeline for the randomly selected abstracts from the validation dataset
pipe_summaries = [0]*5
generated_summaries = [0]*5
j = 0

for i in random_numbers:
  pipe_summaries[j] = pipe(hydro["validation"][i]["Abstract"])
  generated_summaries[j] = pipe_summaries[j][0]["summary_text"]
  j += 1

Your max_length is set to 142, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)


In [ ]:
# Evaluation Before Finetuning The Model
from rouge import Rouge

rouge = Rouge()

scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)
df_scores = pd.DataFrame(scores)

print("ROUGE Scores:")
print(df_scores)

ROUGE Scores:
    rouge-1   rouge-2   rouge-l
r  0.453633  0.314587  0.433704
p  0.394758  0.247466  0.373241
f  0.418367  0.273299  0.397686




---



# Finetuning The Model

In [ ]:
from datasets import load_dataset
from transformers import pipeline

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

In [ ]:
# Setting the device for computation (GPU or CPU)
device = 'gpu'

# Specifying the pre-trained model checkpoint to be used
model_ckpt = 'facebook/bart-large-cnn'

# Loading the tokenizer associated with the pre-trained BART model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Loading the pre-trained BART model for sequence-to-sequence tasks
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Defining a function to generate features for training data batches
def get_feature(batch):
    # Tokenizing abstracts and summaries using the tokenizer
    encodings = tokenizer(batch['Abstract'], text_target=batch['Summary'],
                          max_length=1024, truncation=True)

    # Extracting input_ids, attention_mask, and labels from the tokenized encodings
    encodings = {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': encodings['labels']
    }

    return encodings

In [ ]:
# Applying the get_feature function to map over the entire 'hydro' dataset in batches
hydro_pt = hydro.map(get_feature, batched=True)

Map:   0%|          | 0/315 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
hydro_pt

DatasetDict({
    train: Dataset({
        features: ['Id', 'Abstract', 'Summary', 'Unnamed: 3', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 315
    })
    validation: Dataset({
        features: ['Id', 'Abstract', 'Summary', 'Unnamed: 3', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['Id', 'Abstract', 'Summary', 'Unnamed: 3', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [ ]:
# Setting the format of the 'hydro_pt' dataset to PyTorch tensors with specified columns
hydro_pt.set_format(type='torch', columns=['input_ids', 'labels', 'attention_mask'])

In [ ]:
# Importing the DataCollatorForSeq2Seq class from the transformers library
from transformers import DataCollatorForSeq2Seq

# Creating a data collator for sequence-to-sequence tasks using the tokenizer and model
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
# Importing the TrainingArguments and Trainer classes from the transformers library
from transformers import TrainingArguments, Trainer

# Specifying the training arguments for training the model
training_args = TrainingArguments(
    output_dir='bart_hydro',
    num_train_epochs=10,
    warmup_steps=500,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

# Initializing the Trainer with the specified arguments and data collator
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    train_dataset=hydro_pt['train'],
    eval_dataset=hydro_pt['validation']
)

In [ ]:
# Training the model
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=40, training_loss=0.7572186708450317, metrics={'train_runtime': 472.9219, 'train_samples_per_second': 6.661, 'train_steps_per_second': 0.085, 'total_flos': 1984845602660352.0, 'train_loss': 0.7572186708450317, 'epoch': 8.10126582278481})

In [ ]:
# Saving the trained model to the specified directory
trainer.save_model('./bart_hydro_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


---
---



## Extracting Information from PDF

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

def extract_text_with_pyPDF(PDF_File):

    pdf_reader = PdfReader(PDF_File)

    raw_text = ''

    for i, page in enumerate(pdf_reader.pages):

        text = page.extract_text()
        if text:
            raw_text += text

    return raw_text

In [ ]:
text_with_pyPDF = extract_text_with_pyPDF("hydroPDF1.pdf")
t = str(text_with_pyPDF)
print(t)

See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/8618603
Hydrogen storage methods
Article    in  The Scienc e of Nat ure · April 2004
DOI: 10.1007/s00114-004-0516- x · Sour ce: PubMed
CITATIONS
852READS
40,067
1 author:
Andr eas Z üttel
École P olyt echnique F édér ale de Lausanne
273 PUBLICA TIONS    25,360  CITATIONS    
SEE PROFILE
All c ontent f ollo wing this p age was uplo aded b y Andr eas Z üttel on 04 June 2014.
The user has r equest ed enhanc ement of the do wnlo aded file.Naturwissenschaften (2004) 91:157–172
DOI 10.1007/s00114-004-0516-x
REVIEW
Andreas Z/C252ttel
Hydrogen storage methods
Published online: 17 March 2004
/C23 Springer-Verlag 2004
Abstract Hydrogen exhibits the highest heating value
per mass of all chemical fuels. Furthermore, hydrogen isregenerative and environmentally friendly. There are tworeasons why hydrogen is not the major fuel of today’senergy consumption. First of all, hydrogen is ju

In [ ]:
# Removing References from the text data


def remove_references(text, references_keyword):
    # Find the index where the references section starts
    references_index = text.find(references_keyword)

    # If the references keyword is found, extract the text before it
    if references_index != -1:
        return text[:references_index]
    else:
        return text


# Keyword indicating the start of the references section
references_keyword = "References"

# Remove everything after the references section
t = remove_references(t, references_keyword)

Turning the large PDF text into smaller chunks

In [ ]:
def chunk_text(text, chunk_size):
    # Initialize an empty list to store text chunks
    chunks = []

    # Calculate the total number of chunks
    total_chunks = len(text) // chunk_size + (1 if len(text) % chunk_size != 0 else 0)

    # Split the text into chunks
    for i in range(total_chunks):
        start_index = i * chunk_size
        end_index = (i + 1) * chunk_size
        chunk = text[start_index:end_index]
        chunks.append(chunk)

    return chunks

# Chunk size
chunk_size = 3000

# Split the text into chunks
text_chunks = chunk_text(t, chunk_size)

# Print the number of chunks
print("Number of chunks:", len(text_chunks))

# Print the first chunk
print("First chunk:", text_chunks[0])

print("\n\n")

# Print the second chunk
print("Second chunk:", text_chunks[1])

# Store these chunks in a data structure for later use

Number of chunks: 25
First chunk: See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/8618603
Hydrogen storage methods
Article    in  The Scienc e of Nat ure · April 2004
DOI: 10.1007/s00114-004-0516- x · Sour ce: PubMed
CITATIONS
852READS
40,067
1 author:
Andr eas Z üttel
École P olyt echnique F édér ale de Lausanne
273 PUBLICA TIONS    25,360  CITATIONS    
SEE PROFILE
All c ontent f ollo wing this p age was uplo aded b y Andr eas Z üttel on 04 June 2014.
The user has r equest ed enhanc ement of the do wnlo aded file.Naturwissenschaften (2004) 91:157–172
DOI 10.1007/s00114-004-0516-x
REVIEW
Andreas Z/C252ttel
Hydrogen storage methods
Published online: 17 March 2004
/C23 Springer-Verlag 2004
Abstract Hydrogen exhibits the highest heating value
per mass of all chemical fuels. Furthermore, hydrogen isregenerative and environmentally friendly. There are tworeasons why hydrogen is not the major fuel of today’senergy consump


We'll start by summarizing each smaller section, and then we'll use those summaries to create an overall summary of the paper.

In [ ]:
from transformers import pipeline

# Load the BART model for summarization
barto = pipeline("summarization", model="bart_hydro_model")

In [ ]:
n = len(text_chunks)
summ_chunks = [0] * n

for i in range(len(text_chunks)):
  summ_chunks[i] = barto(text_chunks[i], max_length=100, min_length=50)
  print(i, end=" ")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 

In [ ]:
for i in range(25):
  print(f"{summ_chunks[i][0]['summary_text']}")


Hydrogen exhibits the highest heating value per mass of all chemical fuels. The most common storage systems are high-pressure gas cylinders with a maximum pressure of 20 MPa (200 bar) New light-weight composite cylinders have been developed which are able to withstand pressures up to 80 MPa(800 bar) and therefore the hydrogen gas can reach avolumetric density of 36 kg/m/C1593, approximately half as much as in its liquid state.
Hydrogen can also be stored indirectly in reactive metals such as Li, Na, Al or Zn. These metals easily react with water to the corresponding hydroxide and liberatethe hydrogen from the water. Since water is the product ofthe combustion of hydrogen with either oxygen or air, it can be recycled in a closed loop and react with the metal.
Carbon dioxide is a greenhouse gas and causes an increase in the average temperature on earth. The solubility of carbon dioxide decreases with theincreasing temperature of water by approximately 3%/K. The chemical energy per mass o

In [ ]:
summ = ''
for i in range(n):
  summ += summ_chunks[i][0]['summary_text'] + " "

In [ ]:
# Split the text into chunks
text1_chunks = chunk_text(summ, 1700)

In [ ]:
n1 = len(text1_chunks)
summ1_chunks = [0] * n1

for i in range(n1):
  summ1_chunks[i] = barto(text1_chunks[i], max_length=20, min_length=10)
  print(i, end=" ")

0 1 2 3 4 5 

In [ ]:
print("Overall Summary: \n")
for i in range(n1):
  print(f"{summ_chunks[i][0]['summary_text']}")
  print("\n")

Overall Summary: 

Hydrogen exhibits the highest heating value per mass of all chemical fuels. The most common storage systems are high-pressure gas cylinders with a maximum pressure of 20 MPa (200 bar) New light-weight composite cylinders have been developed which are able to withstand pressures up to 80 MPa(800 bar) and therefore the hydrogen gas can reach avolumetric density of 36 kg/m/C1593, approximately half as much as in its liquid state.


Hydrogen can also be stored indirectly in reactive metals such as Li, Na, Al or Zn. These metals easily react with water to the corresponding hydroxide and liberatethe hydrogen from the water. Since water is the product ofthe combustion of hydrogen with either oxygen or air, it can be recycled in a closed loop and react with the metal.


Carbon dioxide is a greenhouse gas and causes an increase in the average temperature on earth. The solubility of carbon dioxide decreases with theincreasing temperature of water by approximately 3%/K. The che

---
---



# Evaluation Of The Model After Finetuning

In [ ]:
# Evaluation after finetuning the model

import random
start, end = 0, 100

random_numbers = [random.randint(start, end) for _ in range(5)]
random_numbers

[81, 14, 3, 94, 35]

In [ ]:
reference_summaries = [0]*5
j = 0

for i in random_numbers:
  reference_summaries[j] = hydro["validation"][i]["Summary"]
  j += 1

In [ ]:
trained_summaries = [0]*5
pipe_summaries = [0]*5
j = 0

for i in random_numbers:
  pipe_summaries[j] = pipe(hydro["validation"][i]["Abstract"])
  trained_summaries[j] = pipe_summaries[j][0]["summary_text"]
  j += 1

Your max_length is set to 142, but your input_length is only 139. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


In [ ]:
from rouge import Rouge

rouge = Rouge()

scores = rouge.get_scores(trained_summaries, reference_summaries, avg=True)
df_scores = pd.DataFrame(scores)

print("ROUGE Scores:")
print(df_scores)

ROUGE Scores:
    rouge-1   rouge-2   rouge-l
r  0.574513  0.420014  0.559107
p  0.475495  0.326378  0.461521
f  0.509293  0.354605  0.494945


# Storing it to hugging face

---



---



In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [ ]:
barto.push_to_hub("my-BART-Hydrogen-Model"),

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

(CommitInfo(commit_url='https://huggingface.co/Sharukesh/my-BART-Hydrogen-Model/commit/63372fcfebcd5fd6d2ff1d8e5460368bbf94f646', commit_message='Upload SummarizationPipeline', commit_description='', oid='63372fcfebcd5fd6d2ff1d8e5460368bbf94f646', pr_url=None, pr_revision=None, pr_num=None),)

In [ ]:
tokenizer.push_to_hub("my-BART-Hydrogen-Model"),

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

(CommitInfo(commit_url='https://huggingface.co/Sharukesh/my-BART-Hydrogen-Model/commit/a672c098362ad8a356805ce1ebf5e6a716b65bb2', commit_message='Upload tokenizer', commit_description='', oid='a672c098362ad8a356805ce1ebf5e6a716b65bb2', pr_url=None, pr_revision=None, pr_num=None),)

# checking the saved model

In [ ]:
from transformers import pipeline

# Load the BART model for summarization
barto = pipeline("summarization", model="Sharukesh/my-BART-Hydrogen-Model")

generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
n = len(text_chunks)
summ_chunks = [0] * n

for i in range(len(text_chunks)):
  summ_chunks[i] = barto(text_chunks[i], max_length=100, min_length=50)
  print(i, end=" ")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 

In [ ]:
for i in range(25):
  print(f"{summ_chunks[i][0]['summary_text']}")

Hydrogen exhibits the highest heating value per mass of all chemical fuels. The most common storage systems are high-pressure gas cylinders with a maximum pressure of 20 MPa (200 bar) New light-weight composite cylinders have been developed which are able to withstand pressures up to 80 MPa(800 bar) and therefore the hydrogen gas can reach avolumetric density of 36 kg/m/C1593, approximately half as much as in its liquid state.
Hydrogen can also be stored indirectly in reactive metals such as Li, Na, Al or Zn. These metals easily react with water to the corresponding hydroxide and liberatethe hydrogen from the water. Since water is the product ofthe combustion of hydrogen with either oxygen or air, it can be recycled in a closed loop and react with the metal.
Carbon dioxide is a greenhouse gas and causes an increase in the average temperature on earth. The solubility of carbon dioxide decreases with theincreasing temperature of water by approximately 3%/K. The chemical energy per mass o

In [ ]:
summ = ''
for i in range(n):
  summ += summ_chunks[i][0]['summary_text'] + " "

In [ ]:
# Split the text into chunks
text1_chunks = chunk_text(summ, 1700)

In [ ]:
n1 = len(text1_chunks)
summ1_chunks = [0] * n1

for i in range(n1):
  summ1_chunks[i] = barto(text1_chunks[i], max_length=20, min_length=10)
  print(i, end=" ")

0 1 2 3 4 5 

In [ ]:
print("Overall Summary: \n")
for i in range(n1):
  print(f"{summ_chunks[i][0]['summary_text']}")
  print("\n")

Overall Summary: 

Hydrogen exhibits the highest heating value per mass of all chemical fuels. The most common storage systems are high-pressure gas cylinders with a maximum pressure of 20 MPa (200 bar) New light-weight composite cylinders have been developed which are able to withstand pressures up to 80 MPa(800 bar) and therefore the hydrogen gas can reach avolumetric density of 36 kg/m/C1593, approximately half as much as in its liquid state.


Hydrogen can also be stored indirectly in reactive metals such as Li, Na, Al or Zn. These metals easily react with water to the corresponding hydroxide and liberatethe hydrogen from the water. Since water is the product ofthe combustion of hydrogen with either oxygen or air, it can be recycled in a closed loop and react with the metal.


Carbon dioxide is a greenhouse gas and causes an increase in the average temperature on earth. The solubility of carbon dioxide decreases with theincreasing temperature of water by approximately 3%/K. The che



---

